In [19]:
from pathlib import Path
import sys
import cv2
import numpy as np
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from keras.layers import Conv2D, Conv2DTranspose
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import concatenate
from keras.models import Model
from keras.optimizers import Adam

# Training UNET

## Importing the necessary functions

In [20]:
sys.path.append("..") # fix

In [21]:
from training.simple.conv2d_block import conv2d_block
from training.simple.read_dataset import read_dataset

In [22]:
class TrackProgress(Callback):
    def on_epoch_end(self, epoch, logs=None):
        out_progress_folder = Path("progress/best")
        if epoch == 0:
            out_progress_folder.mkdir(parents=True, exist_ok=True)
        if epoch % 10 == 0:
            _images = images_val[::16, :, :].copy()
            _masks = masks_val[::16, :, :].copy()

            n_tests = _images.shape[0]
            _pred_masks = model.predict(_images)

            _images *= 255.0
            _masks *= 255.0
            _pred_masks *= 255.0

            for i in range(n_tests):
                _img = _images[i, :, :, 0].astype(np.uint8)
                _mask = _masks[i, :, :, 0].astype(np.uint8)
                _pred_mask = _pred_masks[i, :, :, 0].astype(np.uint8)
                _out_img = np.hstack((_img, _mask, _pred_mask))
                cv2.imwrite(
                    str(
                        Path(
                            out_progress_folder,
                            f"progress_img_ex_{i:03d}_{epoch:05d}.jpg",
                        )
                    ),
                    _out_img,
                )

## Defining the path of the dataset

In [23]:
dataset_folder = Path(r"/home/higorem/DeepFlowImaging/UNET/examples/dataset_UNET")

## Defining hyperparameters

In [24]:
window_size = 64

# number of iterations of all the training data in one cycle for training the model
EPOCHS = 5000

# number of epochs after the loss of the model get lower as possible
PATIENCE = 500

# number of samples
BATCH_SIZE = 128


## Reading the training dataset

In [25]:
images_train, masks_train = read_dataset(
    dataset_folder, window_size=window_size, subset="Training"
)

## Reading the validation dataset

In [26]:
images_val, masks_val = read_dataset(
    dataset_folder, window_size=window_size, subset="Validation"
)

## Shuffling the training dataset

In [27]:
N_dataset = images_train.shape[0]
shuffle = np.arange(N_dataset)
np.random.seed(13)
np.random.shuffle(shuffle)
images_train = images_train[shuffle, :, :, :]
masks_train = masks_train[shuffle, :, :, :]

## Setting the network architecture

In [28]:
input_img = Input((window_size, window_size, 1), name="img")

c1 = conv2d_block(input_img, 8, kernel_size=5, batchnorm=True)
p1 = MaxPooling2D((4, 4))(c1)

c2 = conv2d_block(p1, 8, kernel_size=5, batchnorm=True)
p2 = MaxPooling2D((4, 4))(c2)

c3 = conv2d_block(p2, 8, kernel_size=5, batchnorm=True)

u4 = Conv2DTranspose(8, kernel_size=5, strides=(4, 4), padding="same")(c3)
u4 = concatenate([u4, c2])
c4 = conv2d_block(u4, 8, kernel_size=5, batchnorm=True)

u5 = Conv2DTranspose(8, kernel_size=5, strides=(4, 4), padding="same")(c4)
u5 = concatenate([u5, c1])
c5 = conv2d_block(u5, 8, kernel_size=5, batchnorm=True)

outputs = Conv2D(1, (1, 1), activation="sigmoid")(c5)
model = Model(inputs=[input_img], outputs=[outputs])
model.compile(optimizer=Adam(), loss="binary_crossentropy")

## Training the model

In [29]:
model.summary()

early_stopping = EarlyStopping(
    monitor="val_loss", mode="min", verbose=1, patience=PATIENCE
)

checkpoint = ModelCheckpoint(
    "UNET_best.keras", verbose=1, save_best_only=True, monitor="val_loss", mode="auto"
)

model.fit(
    images_train,
    masks_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(images_val, masks_val),
    callbacks=[checkpoint, early_stopping, TrackProgress()],
)

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ img (InputLayer)    │ (None, 64, 64, 1) │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_89 (Conv2D)  │ (None, 64, 64, 8) │        208 │ img[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64, 8) │         32 │ conv2d_89[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_81       │ (None, 64, 64, 8) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_16    │ (None, 16, 16, 8) │          0 │ activation_81[0]… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_91 (Conv2D)  │ (None, 16, 16, 8) │      1,608 │ max_pooling2d_16… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16, 8) │         32 │ conv2d_91[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_83       │ (None, 16, 16, 8) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_17    │ (None, 4, 4, 8)   │          0 │ activation_83[0]… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_93 (Conv2D)  │ (None, 4, 4, 8)   │      1,608 │ max_pooling2d_17… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 4, 4, 8)   │         32 │ conv2d_93[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_85       │ (None, 4, 4, 8)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_16 │ (None, 16, 16, 8) │      1,608 │ activation_85[0]… │
│ (Conv2DTranspose)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_16      │ (None, 16, 16,    │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 16)               │            │ activation_83[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_95 (Conv2D)  │ (None, 16, 16, 8) │      3,208 │ concatenate_16[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16, 8) │         32 │ conv2d_95[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_87       │ (None, 16, 16, 8) │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_17 │ (None, 64, 64, 8) │      1,608 │ activation_87[0]… │
│ (Conv2DTranspose)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 13,225 (51.66 KB)

 Trainable params: 13,145 (51.35 KB)

 Non-trainable params: 80 (320.00 B)

Epoch 1/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.7929
Epoch 1: val_loss improved from inf to 0.60973, saving model to UNET_best.keras
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 363ms/step - loss: 0.7895 - val_loss: 0.6097
Epoch 2/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.6819
Epoch 2: val_loss improved from 0.60973 to 0.49304, saving model to UNET_best.keras
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 196ms/step - loss: 0.6792 - val_loss: 0.4930
Epoch 3/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.6055
Epoch 3: val_loss improved from 0.49304 to 0.40933, saving model to UNET_best.keras
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - loss: 0.6037 - val_loss: 0.4093
Epoch 4/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.5522
Epoch 4: val_loss improved from 0.40933 to 0.35216, saving model to UNET_best.keras
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - loss: 0.5514 - val_loss: 0.3522
Epoch 5/5000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.5146
Epoch 5